# Experiment 04: Amazon Planet

This experiment uses the data from the Kaggle competition [Planet: Understanding the Amazon from Space](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/leaderboard). Here we use a pretrained ResNet50 model to generate the features from the dataset.

In [7]:
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import seaborn
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
import pkg_resources
from libs.planet_kaggle import read_images, chunks, labels_from, enrich_with_feature_encoding, get_file_count, to_multi_label_dict
from libs.timer import Timer
from libs.utils import get_number_processors
import warnings
from libs.loaders import load_planet_kaggle

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

warnings.filterwarnings("ignore", category=DeprecationWarning) 
% matplotlib inline
% load_ext autoreload
% autoreload 2

System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
random_seed=42

In [10]:
%env MOUNT_POINT=/mnt/data

env: MOUNT_POINT=/mnt/data


The images are loaded and featurised using a pretrained ResNet50 model available from Keras

In [11]:
X_train, y_train, X_test, y_test = load_planet_kaggle()

INFO:libs.loaders:Reading in labels

Featurising training images:   0%|          | 0/1094.0 [00:00<?, ?it/s]
Featurising training images: 100%|██████████| 1094/1094.0 [09:04<00:00,  1.88it/s]
Featurising validation images: 100%|██████████| 172/172.0 [01:25<00:00,  2.41it/s]


## XGBoost vs LightGBM benchmark

We will compare both libraries on speed and preformance.

In [13]:
number_processors = get_number_processors()

We will use a one-v-rest. So each classifier will be responsible for determining whether the assigned tag applies to the image

In [14]:
def train_and_validate(model, train_features, train_labels, validation_features):
    n_classes = train_labels.shape[1]
    y_val_pred = np.zeros((validation_features.shape[0], n_classes))
    time_results = defaultdict(list)
    for class_i in tqdm(range(n_classes)):
        with Timer() as t:
            model.fit(train_features, train_labels[:, class_i])
        time_results['train_time'].append(t.interval)
        
        with Timer() as t:
            y_val_pred[:, class_i] = model.predict(validation_features)
        time_results['test_time'].append(t.interval)
        
    return y_val_pred, time_results

In [15]:
metrics_dict = {
    'Accuracy': accuracy_score,
    'Precision': lambda y_true, y_pred: precision_score(y_true, y_pred, average='samples'),
    'Recall': lambda y_true, y_pred: recall_score(y_true, y_pred, average='samples'),
    'F1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='samples'),
}

def classification_metrics(metrics, y_true, y_pred):
    return {metric_name:metric(y_true, y_pred) for metric_name, metric in metrics.items()}

In [16]:
results_dict = dict()

In [19]:
xgb_model = XGBClassifier(max_depth=7, 
                         learning_rate=0.1, 
                         scale_pos_weight=2,
                         n_estimators=300,
                         gamma=0.1,
                         min_child_weight=1,
                         reg_lambda=1,
                         subsample=1,
                         n_jobs=number_processors,
                         random_state=random_seed) 

TypeError: __init__() got an unexpected keyword argument 'random_state'

In [23]:
validation_features.shape

(0,)

In [ ]:
y_val_pred, timing_results = train_and_validate(xgb_model, train_features, y_train, validation_features)

In [ ]:
results_dict['xgb']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_val, 
                                          threshold_prediction(y_val_pred, threshold=0.1)) 
}

In [ ]:
xgb_hist_model = XGBClassifier(max_depth=0, 
                              learning_rate=0.1, 
                              scale_pos_weight=2,
                              n_estimators=300,
                              gamma=0.1,
                              min_child_weight=1,
                              reg_lambda=1,
                              subsample=1,
                              max_leaves=2**7,
                              grow_policy='lossguide',
                              tree_method='hist',
                              n_jobs=number_processors,
                              random_state=random_seed)

In [ ]:
y_val_pred, timing_results = train_and_validate(xgb_hist_model, train_features, y_train, validation_features)

In [ ]:
results_dict['xgb_hist']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_val, 
                                          threshold_prediction(y_val_pred, threshold=0.1)) 
}

In [ ]:
lgbm_model = LGBMClassifier(num_leaves=2**7, 
                           learning_rate=0.1, 
                           scale_pos_weight=2,
                           n_estimators=300,
                           min_split_gain=0.1,
                           min_child_weight=1,
                           reg_lambda=1,
                           subsample=1,
                           nthread=number_processors,
                           seed=random_seed

In [ ]:
y_val_pred, timing_results = train_and_validate(lgbm_model, train_features, y_train, validation_features)

In [ ]:
results_dict['lgbm']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_val, 
                                          threshold_prediction(y_val_pred, threshold=0.1)) 
}

In [ ]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))